# Applying TFIDF Vectors and t-SNE to Subreddit Content
While researching for this project, I discovered a whole lot of subreddit mappings and visualaztions out there. Most of them employ user engagement as a metric for modularity detection. (Insert Sources). Since this project is based on breaking free of traditional user interest-patterns, I decided to quantify subreddit differences by content alone, in much the same way as Andrej Kaparthy does in [this blog post](http://karpathy.github.io/2014/07/02/visualizing-top-tweeps-with-t-sne-in-Javascript/).

## Collecting Data
Using the previously mentioned post as a rough guide, I started implementing the community difference analysis. Initial analysis, I simply downloaded comment text from a large handful of default subreddits, with the code saved in '/download.py'. Basically I just save a file for each sub with concatenated raw comment text, and a list of the files (for scikit to use later on).

In [4]:
import pickle
files_store = open('data/files.pickle','rb')
files_list = pickle.load(files_store)
print(files_list)

['gadgets.txt', 'sports.txt', 'gaming.txt', 'pics.txt', 'worldnews.txt', 'videos.txt', 'AskReddit.txt', 'aww.txt', 'funny.txt', 'news.txt', 'movies.txt', 'blog.txt', 'books.txt', 'history.txt', 'food.txt', 'philosophy.txt', 'Jokes.txt', 'Art.txt', 'DIY.txt', 'space.txt', 'Documentaries.txt', 'askscience.txt', 'nottheonion.txt', 'todayilearned.txt', 'personalfinance.txt', 'gifs.txt', 'listentothis.txt', 'IAmA.txt', 'announcements.txt', 'TwoXChromosomes.txt', 'creepy.txt', 'nosleep.txt', 'GetMotivated.txt', 'WritingPrompts.txt', 'LifeProTips.txt', 'EarthPorn.txt', 'explainlikeimfive.txt', 'Showerthoughts.txt', 'Futurology.txt', 'photoshopbattles.txt', 'mildlyinteresting.txt', 'dataisbeautiful.txt', 'tifu.txt', 'OldSchoolCool.txt', 'UpliftingNews.txt', 'InternetIsBeautiful.txt', 'science.txt']
